In [4]:
import os
import requests
import re
from bs4 import BeautifulSoup
import lxml
import os.path
import time
import warnings
warnings.filterwarnings("ignore")
import pickle
import pandas as pd
from requests.exceptions import ProxyError,ConnectionError,Timeout,HTTPError,ChunkedEncodingError
import itertools
import multiprocessing
import Ipynb_importer
from IPpool import GetHeaders
from IPpool import Ip_pool
import random
import copy

In [2]:
#***************自定义异常类**************************
class MyException(Exception):
    def __init__(self,message):
        Exception.__init__(self)
        self.message=message

In [1]:
class Crawl_common(object):
    '''
    这是用来爬取大众点评的共有包，包含了以下的函数。
    
    函数
    -------
    
    1. requests_visit_url(self,url,is_use_ip =False,timeout=3)
        函数功能：访问特定的url，带有ip池的功能
          
    2.  save_cache(self,result,filename):
        功能：保留缓存
    
    3. read_cache(self,filename):
        功能：读取数据
        
    4. get_spc_path(self,father_path,child_path)
        功能：切换一个路径或者在路径下新建一个子路径
        
    5.get_chunk(self,n,adr_ls=None)
        功能：将数组分为n组，可以用来分组地去跑函数
        
    '''
    def __init__(self,test_url = 'http://www.dianping.com/shop/95562383',more=False):
        '''
        功能：生成ip池
        
        参数
        ----
        more:bool 类型，more等于True的时候，采取4个网站的ip做ip池
        '''
        self.test_url = test_url
        global ip_pool
        global bad_ip
        print("获取ip池----------------------")
        if more:
            print("获取4个网站的ip")
            ip_url = ['http://www.xicidaili.com/nn/']
            ip_pool= Ip_pool().get_ip_pool(self.test_url,timeout=5)  
        else:
            print("获取1个网站的ip")
            ip_url = ['http://www.xicidaili.com/nn/']
            ip_pool= Ip_pool(ip_url).get_ip_pool(self.test_url,timeout=5)  
    
    #*************带ip池访问url**********************************
    def requests_visit_url(self,url,is_use_ip =True,timeout=5):
        '''
        功能：自带ip池功能来放访问特定的url
        
        参数
        ----
        url：sring，表示特定的ul
        is_use_ip :bool，True表示采用ip池访问
        timeout:int,表示一次访问url的最大超时限制
        
        结果
        ----
        soup:BeautifulSoup对象，可以用来解析网页
        '''
        global ip_pool
        global bad_ip
        #headers = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36"}
        headers = {'User-Agent':random.choice(GetHeaders().user_agent_list)}

        #获取任意一个请求头
        #headers["User-Agent"] = random.choice(GetHeaders().user_agent_list)
        bad_ip = []  #用来装坏的ip
        
        #*********如果不设置ip的话**********************************
        if is_use_ip == False:
            res = requests.get(url,headers = headers,timeout=timeout)  #访问url，不设代理ip访问
            html =  res.content.decode('utf-8')
            soup  =  BeautifulSoup(html,'lxml')
            if soup.title.text == '403 Forbidden' or "有道" in soup.title.text:
                print("ip被禁止了！！！！！！！！！！！！！！！！")
                raise ConnectionError
           
        #************如果设置ip的话，则执行这一段*********************
        else:
            headers = {'User-Agent':random.choice(GetHeaders().user_agent_list)}
            is_continue = True   #设置循环标志
            while is_continue:
            #整理成ip地址的格式
                try:
                    #
                    if len(ip_pool)==len(bad_ip):
                        ip_url = ['http://www.xicidaili.com/nn/']
                        #self.test_url = 'http://www.dianping.com/shop/95562383'
                        ip_pool= Ip_pool().get_ip_pool(self.test_url,timeout=2)
                        prox_ip = random.choice(ip_pool)
                        bad_ip = []
                        
                    #******随机生成一个ip*******
                    prox_ip = random.choice(ip_pool)
                    http = prox_ip[0]
                    ip = prox_ip[1]
                    proxies={http:ip}
                    
                    #*******用代理ip访问********
                    res = requests.get(url,headers = headers,proxies = proxies,timeout=timeout) #设代理ip访问
                    html =  res.content.decode('utf-8')
                    soup  =  BeautifulSoup(html,'lxml')
                    
                    
                    #********如果返回网页被禁止的情况，触发异常,否则返回正常**********
                    if  soup.title.text == '403 Forbidden' or "有道" in soup.title.text:
                        print("ip被禁止了!!!!!!!!!!!!!!!!!!!!")
                        raise ConnectionError
                    else:
                        is_continue = False

                except HTTPError as e:
                    bad_ip.append(prox_ip)
                    print("断网")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)


                except Timeout as e:
                    bad_ip.append(prox_ip)
                    print("超时")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)


                except ConnectionError as e:
                    bad_ip.append(prox_ip)
                    print("访问被拒")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)
                    
                except AttributeError as e:
                    bad_ip.append(prox_ip)
                    print("----------")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)
                    
                except ChunkedEncodingError as e:
                    bad_ip.append(prox_ip)
                    print("ChunkedEncodingError")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)

        return soup
    

    #**************缓存结果******************
    def save_cache(self,result,filename):
        '''
        功能：保留缓存
        
        参数
        -------
        result:任何类型，保存的结果。
        filename:字符串，保存的路径。
        
        返回值
        -------
        无
        '''
        with open(filename,'wb') as f1:
            pickle.dump(result,f1)
            
    #---------加载缓存-----------------------
    def read_cache(self,filename):
        '''
        功能：读取数据
        
        参数
        ------
        filename:字符串，文件的路径
        
        返回值：
        ----
        result：任何对象
        '''
        with open(filename,'rb')as f1:
            result = pickle.load(f1)
        return result
    
    #----------切换路径-------------------
    def get_spc_path(self,father_path,child_path):
        '''
        功能：切换路径，创建文件夹
        
        参数
        -----
        father_path:字符串，绝对路径的父目录
        child_path：字符串，绝对路径的子目录
        '''
        
        os.chdir(father_path)
        if os.path.exists(child_path):
            os.chdir(child_path)
        else:
            os.mkdir(child_path)
            os.chdir(child_path)
        print("done!!")

    def get_chunk(self,n,adr_ls=None):
        '''
        功能：将数组分为n组，可以用来分组地去跑函数
        
        参数：
        ----
        n:int型，表示将数组分为n组
        adr_ls:list，表示要被切分的数组
        '''
        new_adr_ls = copy.deepcopy(adr_ls)
        adr_chunk = [new_adr_ls[i:i + n] for i in range(0, len(new_adr_ls), n)]
        return adr_chunk
        
        
#     #---------定位一级分类，爬取二级分类------------------
#     def classify(self,city_url,city,big_class):
#         soup = requests_visit_url(city_url)
#         clf_ls = []
#         for i in soup.select("#classfy a"):
#             clf_url = i["href"] #分类网址
#             clf_tl = i.text.strip() # 分类标题
#             clf_ls.append([big_class,city,clf_tl,clf_url])
#         return clf_ls
    
#     #定位二级分类，爬取区县（行政区）
#     def get_region(self,class_ls=None):
#         url = class_ls[-1]  #url
#         soup = requests_visit_url(url) #访问
#         region_ls = [] #地区的列表
#         for i in soup.select("#region-nav a"):
#             if i.text.strip() != "不限":
#                 region_url = i["href"] 
#                 region_tl = i.text.strip()
#                 region = class_ls[:-1]
#                 region.extend([region_tl,region_url])
#                 region_ls.append(region)
#         return region_ls
    
#     #-----------定位了区县，爬取商圈-----------
#     def get_circle(self,region_ls=None):
#         url = region_ls[-1]   #url网址
#         soup = self.requests_visit_url(url) #访问BeautifulSoup
#         circle_ls =[]             
#         for i in soup.select("#region-nav-sub a"):  #商圈信息
#             if i.span.text.strip()!="不限":   #剔除掉不限的url
#                 circle_url = i["href"]       #商圈url
#                 circle_tit = i.span.text.strip()  #商圈名
#                 circle = region_ls[:-1] 
#                 circle.extend([circle_tit,circle_url]) 
#                 circle_ls.append(circle)

#         #如果没有商圈的话，就拿原来的url当作是商圈的url
#         if circle_ls==[]:
#             print(region_ls[-2]+"没有子分类！！")
#             region_ls.insert(4,"没有商圈")
#             circle_ls.append(region_ls)

#         return circle_ls
    
#     #---------------获取商家具体信息----------------------
#     #获取详情层的数据，如星级用户、浏览人数、均价
#     def get_info(store_ls=None,prox_ip=None):

#         #首先先保证导入的数据不为空
#         if store_ls !=[]:
#             url = store_ls[-1] #详情信息表
#             soup = requests_visit_url(url,prox_ip)
#             print(soup)
#             try:
#                 start = soup.select(".mid-rank-stars")[0]["title"]       #星级用户
#             except Exception as e:
#                 print("start",e)
#                 start = ''

#             try:
#                 review_count = soup.select("#reviewCount")[0].text  #浏览人数
#             except Exception as e:
#                 review_count = ''
#                 print("review_coun",e)

#             try:
#                 avgprice = soup.select("#avgPriceTitle")[0].text   #均价
#             except Exception as e:
#                 avgprice = ''
#                 print("avgprice",e)

#             #评分系统
#             try:
#                 score= []
#                 for j in soup.select("#comment_score .item"):
#                     score.append(j.text)
#                 score = "|".join(score)

#             except Exception as e:
#                 score = ""
#                 print("score",e)

#             try:   
#                 address = soup.select(".address .item")[0].text.strip() #地址
#             except Exception as e:
#                 address = ''
#                 print("address出错了",e)

#             #电话号码
#             try:
#                 tel =[]
#                 for i in soup.select(".tel .item"):
#                     tel.append(i.text)
#                 tel = "|".join(tel)

#             except:
#                 tel=""
#                 print("tel出错了",e)

#             #路径
#             try:
#                 path = []
#                 for i in soup.select(".breadcrumb a"):
#                     path.append(i.text.strip())
#                 path ='>'.join(path)
#             except Exception as e:
#                 print("path出错了",e)
#                 path =''

#             info = store_ls[:]
#             info.extend([start,review_count,avgprice,score,address,tel,path])
#         return info
